# Import 2D material from JARVIS database

Import a structure from the NIST JARVIS database by its ID. Use data at <a href="https://figshare.com/articles/dataset/Monolayer_data_for_heterostructure/22344571" target="_blank">JARVIS-DFT</a> to find the ID of the structure you are interested in.

<h2 style="color:green">Usage</h2>

1. Set Input Parameters (e.g. `JARVIS_ID`) below or use the default value
1. Click "Run" > "Run All" to run all cells
1. Wait for the run to complete (it should take ~30 sec first time to install packages). Scroll down to view cell results.

## Methodology

The following happens in the script below:

1. Install `express-py` alongside the necessary packages
2. Import the material from JARVIS database
3. Convert the material to ESSE format
4. Pass the material to the outside runtime

## 1. Set Parameters

In [1]:
# Note: JVASP-670 is an entry for MoTe2
JARVIS_IDS = [
    "JVASP-670",
]

## 2. Install Packages

In [ ]:
from jupyterlite.utils import install_packages

await install_packages("import_material_from_jarvis_db_entry.ipynb")

## 3. Data Processing

### 3.1. Get data and wrap it in dataframe.

In [ ]:
import pandas as pd
from jarvis.db.figshare import data

dft_2d = data("dft_2d")
dft_3d = data("dft_3d")
dataframe_2d = pd.DataFrame(dft_2d)
dataframe_3d = pd.DataFrame(dft_3d)
dataframe= pd.concat([dataframe_2d, dataframe_3d])

### 3.2. Filter data

In [ ]:
import json
from express import ExPrESS

dataframe_entry_for_jarvis_id = dataframe[(dataframe["jid"].isin(JARVIS_IDS))]
dataframe_entry_as_json = dataframe_entry_for_jarvis_id.to_json(orient="records")
jarvis_db_entry = json.dumps(json.loads(dataframe_entry_as_json)[0])


### 3.3. Convert to ESSE format

In [ ]:
kwargs = {
    "structure_string": jarvis_db_entry,
    "cell_type": "original",
    "structure_format": "jarvis-db-entry",
}

handler = ExPrESS("structure", **kwargs)
data = handler.property("material", **kwargs)

### 3.4. Preview the data

In [ ]:
from src.utils import poscar_to_ase
from ase.visualize import view
from ase.io import write
from ase.build import make_supercell
from IPython.display import Image

# Uncomment to see the JSON
# print(json.dumps(data, indent=4))

poscar = handler.parser.jarvis_db_entry_json_to_poscar(jarvis_db_entry)
material = poscar_to_ase(poscar)

# Set the number of unit cell repetition for the structure to make it easier to see
n = 3
view(material, viewer='x3d', repeat=(n,n,n))

# Repeat the structure to make it easier to see
material_repeat = make_supercell(material, [[n,0,0],[0,n,0],[0,0,n]])
write('material.png', material_repeat)

Image(filename='material.png')

## 4. Pass data to the outside runtime

In [ ]:
from jupyterlite.utils import set_data

output_materials = [data]
set_data("materials", output_materials)